# TREC-6
# DistilBERT finetuning

## Librairy

In [2]:
import os
import time
import pickle

import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from transformers import DistilBertTokenizer, DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification, AdamW
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from transformers.data.data_collator import DataCollatorWithPadding

from datasets import load_dataset, Dataset, concatenate_datasets

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Global variables

In [5]:
BATCH_SIZE = 24 # cf. paper Sun et al.
NB_EPOCHS = 4   # cf. paper Sun et al.

In [6]:
# CURRENT_PATH = '~/Results/BERT_finetune' # put your path here
CURRENT_PATH = '/raid/home/jeremiec/Ax_results/DistilBERT_finetune_v2'

In [7]:
RESULTS_FILE = os.path.join(CURRENT_PATH, 'trec-6_results.pkl')
RESULTS_DIR = os.path.join(CURRENT_PATH,'trec-6/')

In [8]:
# CACHE_DIR = '~/Data/huggignface/'         # put your path here
CACHE_DIR = '/raid/home/jeremiec/huggingface_datasets'

## Dataset

In [9]:
# download dataset

raw_datasets = load_dataset('trec', cache_dir=CACHE_DIR)

Using custom data configuration default
Reusing dataset trec (/raid/home/jeremiec/huggingface_datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9)


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['label-coarse', 'label-fine', 'text'],
        num_rows: 5452
    })
    test: Dataset({
        features: ['label-coarse', 'label-fine', 'text'],
        num_rows: 500
    })
})

In [13]:
# tokenize

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column('label-coarse', 'labels') # 'label-fine'
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

train_dataset = tokenized_datasets["train"].shuffle(seed=42)
train_val_datasets = train_dataset.train_test_split(train_size=0.8)

train_dataset = train_val_datasets['train']
val_dataset = train_val_datasets['test']
test_dataset = tokenized_datasets["test"].shuffle(seed=42)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
# get number of labels

num_labels = len(set(train_dataset['labels'].tolist()))
num_labels

6

## Model

#### Model

In [15]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)
model.to(device)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

#### Training

In [16]:
training_args = TrainingArguments(
    
    # output
    output_dir=RESULTS_DIR,          
    
    # params
    num_train_epochs=NB_EPOCHS,               # nb of epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # cf. paper Sun et al.
    learning_rate=2e-5,                       # cf. paper Sun et al.
#     warmup_steps=500,                         # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                         # cf. paper Sun et al.
    weight_decay=0.01,                        # strength of weight decay
    
#     # eval
    evaluation_strategy="steps",              # cf. paper Sun et al.
    eval_steps=50, # 20                       # cf. paper Sun et al.
#     evaluation_strategy='no', # no more evaluation, takes time
    
    # log
    logging_dir=RESULTS_DIR+'logs',  
    logging_strategy='steps',
    logging_steps=50, # 20
    
    # save
    save_strategy='steps',
    save_total_limit=1,
    # save_steps=20, # default 500
    load_best_model_at_end=True,              # cf. paper Sun et al.
    metric_for_best_model='eval_loss'    
)

In [17]:
def compute_metrics(p):
    
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    
    return {"val_accuracy": accuracy}

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # compute_metrics=compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [19]:
results = trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, label-fine.
***** Running training *****
  Num examples = 4361
  Num Epochs = 4
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 728


Step,Training Loss,Validation Loss
50,1.744500,1.622751
100,1.243100,0.697730
150,0.574000,0.395111
200,0.401400,0.309208
250,0.270300,0.291032
300,0.252600,0.243397
350,0.199500,0.264238
400,0.187200,0.221631
450,0.121400,0.237548
500,0.144500,0.219578


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, label-fine.
***** Running Evaluation *****
  Num examples = 1091
  Batch size = 24
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, label-fine.
***** Running Evaluation *****
  Num examples = 1091
  Batch size = 24
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, label-fine.
***** Running Evaluation *****
  Num examples = 1091
  Batch size = 24
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, label-fine.
***** Running Evaluation *****
  Num examples = 1091
  Batch size = 24
The following columns in the eva

In [20]:
training_time = results.metrics["train_runtime"]
training_time_per_epoch = training_time / training_args.num_train_epochs
training_time_per_epoch

13.182125

In [21]:
trainer.save_model(os.path.join(RESULTS_DIR, 'checkpoint_best_model'))

Saving model checkpoint to /raid/home/jeremiec/Ax_results/DistilBERT_finetune_v2/trec-6/checkpoint_best_model
Configuration saved in /raid/home/jeremiec/Ax_results/DistilBERT_finetune_v2/trec-6/checkpoint_best_model/config.json
Model weights saved in /raid/home/jeremiec/Ax_results/DistilBERT_finetune_v2/trec-6/checkpoint_best_model/pytorch_model.bin
tokenizer config file saved in /raid/home/jeremiec/Ax_results/DistilBERT_finetune_v2/trec-6/checkpoint_best_model/tokenizer_config.json
Special tokens file saved in /raid/home/jeremiec/Ax_results/DistilBERT_finetune_v2/trec-6/checkpoint_best_model/special_tokens_map.json


## Results

In [22]:
# load model
model_file = os.path.join(RESULTS_DIR, 'checkpoint_best_model')
finetuned_model = DistilBertForSequenceClassification.from_pretrained(model_file, num_labels=num_labels)
finetuned_model.to(device)
finetuned_model.eval()

# compute test acc
test_trainer = Trainer(finetuned_model, data_collator=DataCollatorWithPadding(tokenizer))
raw_preds, labels, _ = test_trainer.predict(test_dataset)
preds = np.argmax(raw_preds, axis=1)
test_acc = accuracy_score(y_true=labels, y_pred=preds)

# save results
results_d = {}
results_d['accuracy'] = test_acc
results_d['training_time'] = training_time

loading configuration file /raid/home/jeremiec/Ax_results/DistilBERT_finetune_v2/trec-6/checkpoint_best_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version":

In [23]:
results_d

{'accuracy': 0.976, 'training_time': 52.7285}

In [24]:
# save results

with open(RESULTS_FILE, 'wb') as fh:
    pickle.dump(results_d, fh)

In [25]:
# # load results

# with open(RESULTS_FILE, 'rb') as fh:
#     results_d = pickle.load(fh)
    
# results_d